In [ ]:
from io import BytesIO

import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import matplotlib.pyplot as plt
import hvplot.pandas
import matplotlib.gridspec as gridspec
import matplotlib
import holoviews as hv


In [ ]:
cm = 1/2.54  # centimeters in inches

In [ ]:
def get_number_of_rows(dataframe):
    df = pd.read_csv(dataframe)
    return df.shape[0]

In [ ]:
import os
def scandir(event = None):
    path = '../data'
    files = os.listdir(path)
    path_files = [path + '/'+ string for string in files]
    number_of_rows = [get_number_of_rows(dataframe) for dataframe in path_files]
    #dict_values = [(file_name, n_rows) for file_name, n_rows in zip(path_files, number_of_rows) ]
    file_entries = dict(zip(files,path_files))
    file_rows = dict(zip(path_files,number_of_rows))
    return number_of_rows, file_entries, file_rows 
number_of_rows, file_entries, file_rows = scandir()

In [ ]:
def iqr(array):
    q3, q1 = np.percentile(array,[75,25])
    IQR = q3 - q1
    upper_bound = q3 + 1.5*IQR
    lower_bound = q1 - 1.5*IQR
    return lower_bound, upper_bound

In [ ]:
select = pn.widgets.Select(options=file_entries)

@pn.cache
def fetch_data(url):
    df = pd.read_csv(url)
    df.drop('time', axis = 1, inplace = True)
    df.insert(0,'time',df.index.values)
    df = df.fillna(0)
    for column in df.keys():
        lower_bound, upper_bound =iqr(df[column].to_numpy())
        df = df[df[column] > lower_bound ]
        df = df[df[column] < upper_bound ]


    return df




In [ ]:
MB = 1024*1024
MIN_SIZE= 1 * MB
SAVE_PATH = '../data'
file_input = pn.widgets.FileInput(accept='.csv', multiple = True)

def upload_file(event):
    if file_input is not None:
        file_input.save(file_input.filename)


In [ ]:
def get_data(event):
    if file_input is None:
        return_data = None
    else:
        return_data = BytesIO(file_input)
        with open(file_input.filename, "wb") as f:
            f.write('../data/'return_data).getbuffer()

In [ ]:
file_input.param.watch(get_data,'value')

In [ ]:
@pn.cache
def plot(df = None, limb = 'shoulder',direction = 'l', color_0 = 'goldenrod',color_1 = 'coral'):
    data = fetch_data(df)    
    fig  = plt.figure(num=1, clear=True, figsize=(25*cm, 15*cm))
    spec = fig.add_gridspec(ncols=2, nrows=2)
    ax0 = fig.add_subplot(spec[0, :])
    ax0.plot(data[limb + "L" + "angle"],color = '#005bec')
    ax0.plot(data[limb + "R" + "angle"],color = '#ec00c0')
    ax0.set_title(limb + "L" + "angle & "+limb +"R" + "angle")

    ax0.set_facecolor("#e8de8d")

    ax1 = fig.add_subplot(spec[1, 0])
    ax1.set_facecolor("#e8de8d")
    ax1.set_title(limb + "L" + "angle")
    ax1.plot(data[limb + "L" + "angle"],color = '#005bec')
    ax2 = fig.add_subplot(spec[1, 1])
    ax2.set_facecolor("#efd7d7")
    ax2.set_title(limb + "R" + "angle")
    ax2.plot(data[limb + "R" + "angle"], color = '#ec00c0')



    return fig


In [ ]:
@pn.cache
def boxplot(df, axis, direction = 'L', width = 200, height = 300):
    df = pd.read_csv(df)
    axis = axis + direction +'angle'
    boxwhisker = hv.BoxWhisker(df[axis], label=axis)
    return boxwhisker.opts(show_legend=False, width = width,  height = height)

In [ ]:
columns = ['shoulder','elbow','knee','hip']
select_column = pn.widgets.Select(options=columns)
direction = ['l','r']
select_direction =  pn.widgets.Select(options=direction)

main_widget = pn.bind(plot, select,select_column,select_direction)

In [ ]:
boxplot_widget_l = pn.bind(boxplot,select, select_column)
boxplot_widget_r = pn.bind(boxplot,select, select_column ,direction = 'R')

@pn.cache
def boxplot(df = None, limb = 'shoulder',direction = 'l', color_0 = 'goldenrod',color_1 = 'coral'):
    data = fetch_data(df)
    fig1,ax1  = plt.subplots(num=1, figsize=(5*cm, 5*cm))
    ax1.set_title(limb + direction + " angle boxplot")
    ax1.boxplot(data[limb+direction.upper()+"angle"])
    return fig1

boxplot_widget = pn.bind(boxplot, select)
    

## Create the Template

In [ ]:
#Layout using Template
template = pn.template.BootstrapTemplate(
    title='Data analysis for motor disabilities rehabilitation of the upper limbs using augmented reality', 
    sidebar=[pn.pane.Markdown(""), 
             file_input,
             pn.pane.Markdown("## Choose the dataframe"), 
             select_column,
             boxplot_widget_l,
             boxplot_widget_r,
            ],
    main= [pn.Row(main_widget)],
    accent_base_color="#45d8b1",
    header_background="#5608b0",
)
template.show()
template.servable();